<a href="https://colab.research.google.com/github/paulodowd/Webots_2021/blob/master/WLabsheets/L2_LineFollowing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labsheet 2: Line Following

This labsheet has exercises for you to develop two different approaches to using the ground sensor - the simulated sensor we will use to get the simulated robot to follow a line of the arena floor.

- Bang-Bang Controller
- Weighted-Measurement
- Proportional Controller


<hr><br><br><br><br>

## Ground Sensor (Line Sensor)

The Webots simulation of the e-puck has a model of a <a href="http://www.e-puck.org/index.php?option=com_content&view=article&id=17&Itemid=18">real ground sensor</a> developed for the <a href="http://www.e-puck.org/">real e-puck</a>.

<p align="center">
<img src="https://github.com/paulodowd/Webots_2021/blob/master/images/Webots_GroundSensor.png?raw=true">
</p>

The ground sensor works by transmitting a beam of infra-red light, and then measuring the intensity of the reflection.  Different surfaces, such as black and white colour, reflect the light with different intensity.  The intensity of reflected light is a continuous signal, and would be measured by an **`analog-to-digital converter`** - a specialist piece of <a href="https://en.wikipedia.org/wiki/Analog-to-digital_converter">electronic hardware</a>.  

The sensing elements (**`transducers`**) that measure the reflectance would be considered to be an **`analog sensor`**.  The real ground sensor (not simulation) is queried by the microcontroller using the I2C protocol ( 'i'-squared-C, <a href="https://en.wikipedia.org/wiki/I%C2%B2C">"inter-integrated circuit"</a>), which is a method of transferring information digitally.  Therefore, from a system interface perspective, the device as a whole is considered **`digital`**.

<p align="center">
<br>
<img src="https://github.com/paulodowd/Webots_2021/blob/master/images/Webots_GSReflectance.png?raw=true">
<br>
</p>


The <a href="http://www.e-puck.org/index.php?option=com_phocadownload&view=file&id=18:ground-sensor-user-manual&Itemid=38">user manual</a> tells us that the inside-distance between the two sensors on the outside edges is 12mm.  Therefore, using some **`closed-loop feedback control`**, we can keep a line of 12mm or less between the two external sensors.  From this, we can build some line following behaviour.

We can quickly sketch out a plan to produce line following behaviour from the knowledge we have so far.  In the illustration below, there are 3 examples of the robot in different scenarios with respect to the line:

<p align="center">
<br>
<img src="https://github.com/paulodowd/Webots_2021/blob/master/images/Webots_LineFeedback.png?raw=true">
<br>
</p>

We can quickly draft the following logic:

- In **`scenario (A)`**, the central sensor is activated by the black line, and so the robot can continue with it's normal direction of travel.  This is the ideal scenario for line following.  Here, the ideal is to keep the black line between the inactivated outside sensors.  Note that, whilst we (humans) can observe the robot _will_ leave the line, presently the robot does not have this information.

- In **`scenario (B)`**, the robot has veered off the line to the **`left`**, causing the right-most sensor to become activated.  To bring the line back under the centre sensor, the appropriate feedback is to **`turn right`**.

- In **`scenario (C)`**, the robot has veered off the line to the **`right`**, causing the left-most sensor to become activated.  To bring the line back under the centre sensor, the appropriate feedback is to **`turn left`**.

In essence, we can write a controller to keep the black line between the left and right sensors.  Note, there are other ways to utilise this sensor to follow the line.  







## Exercise 1: Ground Sensor (30 minutes)

Feel free to discuss these questions with your peers:

1. Is the ground sensor an **`active`** or **`passive`** sensor?  What informs your answer?

2. Using the logic provided above, what would you expect to observe your robot do if it started off the line (not on the line)?
  
3. What motion would you expect to observe in the robot motion if the feedback-signal was inverted?
  - is there a case where line following can be achieved with an inverted feedback signal?

4. An exceptional case not caught with the above logic would be if all three ground-sensors were activated simultaneously.  This can happen even though the black line is less than 12mm wide.  Under what condition might this occur?

5. What would be appropriate feedback responses for exceptional (non-defined) cases of the sensor activation?

6. If your robot was to calculate a performance score within `loop()` for line-following (a **`metric`**) as it operated:
  - what **`proprioceptive`** information could be used?
  - what **`exteroceptive`** information could be used?

<hr><br><br><br><br>

## Bang-Bang Controller

The most intuitive form of controller we can write for line following is a **`bang-bang`** controller.  A bang-bang controller has this name because the control decisions tend to be sudden and dramatic (bang!), moving between discrete states (bang-bang).  

A consequence of this can be that there is no gradation in behaviour, or the behaviour looks jerky.  Usually, a bang-bang controller is not a good controller to use, but it is a good place to start if you have not programmed before.  It will help to develop a sense of how `conditional statements` can be used to branch code, to change the flow of your program, and so change the observable robot behaviour.  

A bang-bang controller is built using a series of **`if()`** statements.  We will use the `if()` statement to check if a condition is true, and if so, effect a specific behaviour.  

We can interpret the above section and write some **`pseudo-code`**:

```c
if( left_sensor == active_threshold ) {
  // Turn right.

} else if( right_sensor == active_threshold ) {
  // Turn left

} else if ( center_sensor == active_threshold ) {
  // Move forwards

} else {
  // Are there other conditions?
  // What is an appropriate response?

}
```

To implement this code in Webots, you will need to use the built-in functions to both read the ground sensors and to set the appropriate motor velocities.  

From `Labsheet 1`, you should be experienced with commanding the robot to drive forwards/backwards and to rotate.  To read the ground sensor, the template example (`labsheet_x.c`) provides the following code to report the values to the console:

```c
  // Get latest ground sensor readings
  gs_value[0] = wb_distance_sensor_get_value(gs[0]);
  gs_value[1] = wb_distance_sensor_get_value(gs[1]);  
  gs_value[2] = wb_distance_sensor_get_value(gs[2]);

   // Report ground sensor values
  printf("Ground sensor values: \n");
  printf(" 0: %d\n", gs_value[0] );
  printf(" 1: %d\n", gs_value[1] );
  printf(" 2: %d\n", gs_value[2] );
```



## Exercise 2: Bang-Bang Controller (4 hours)

<p align="center">
<img src="https://github.com/paulodowd/Webots_2021/blob/master/images/Webots_StartStraight.png?raw=true">
</p>

1. Is a bang-bang controller **`open-loop`** or **`closed-loop`** control?  What is the difference?

2. Before you write code to command your robot to move, use the graphical user interface to position your robot ground sensor over the black line and record the ground sensor values.
  - Of the three sensors indexed `0,1,2`, which is left, centre and right?
  - What are typical values over a white surface?
  - What are typical values over a black surface?
  - How much **`variance`** is there in these readings?
  - What would be a good value to use as a **`threshold`** for the condition of your if() statements?


3. Implement the discussed bang-bang controller logic to achieve line-following:
  - it is easiest to start the simulation with your robot already placed on a `straight` section of the line.  Check that it can follow a straight line first.
  - it is recommended you start with `slow` or `low` motor velocities.
  - check that your `feedback signal` turns your robot in the appropriate direction.
  - **help**: remember your conditional statement can use:
    - `<`  less than
    - `<=` less than or equal to
    - `==` equal to
    - `>=` greater than or equal to
    - `>`  greater than
    - `!=` not equal to
  - **help**: what is the functional difference between the two code examples immediately below?
  
  ```c
  // Example 1
  if( ) {

  }
  if( ) {

  }

  // Example 2
  if( ) {

  } else if( ) {

  }
  ```

4. Does your robot conduct `turn` and `move fowards` operations seperately?  
  - Can these be integrated so that the robot does not stop moving forwards?
  - How is performance effected?
  - Write down a set of **`task requirements`** where fast forward speed might be desirable for the robotic system.
  - What is the quickest forward speed you can utilise and still achieve reliable line-following?
  - For a selected foward speed, which elements of the line following map can be completed, and which cannot?  Decide upon a set of consistent labels for the elements of the line following map.
  - Decide a discrete list of forward speed intervals to capture the **`failure modes`** you observe.  Are the different forward speeds clearly seperable in their line following performance?

5. Start your robot off the line, and allow it to travel forward to join and follow the line.  Currently, what is the most extreme <a href="https://en.wikipedia.org/wiki/Angle_of_incidence_(optics)">angle of incidence</a> where your controller can still successfully begin line following?
  - if you were to create a results table of different angles when joining the line, how could you quantify the reliability of the controller?

6. What information about the line does the robot have when no sensors are activated?
  - When might this circumstance occur?
  - What would be an appropriate response in this condition?
  - What other information is available to the robot that might be useful?
  
7. Write a function to simply confirm if the robot is on a black line.  The function should report a `true` or `false` value when called.
  - is there a reason to discriminate between which of the sensors is active?  Explain your reasoning, adjust the function if necessary.
  
8. Write a specific function to operate the robot when it is **`initialised`** ("powered on" for the first time) off the line, so that it can successfully join a line at 90 degrees (orthogonal) to the forward direction of travel.
  - Aligning your robot to 45 degrees (approx. `0.785` radians) in the `start box` on the provided arena floor will achieve this starting condition.
  - Decide what your robot should do when it meets an orthogonal line, or how to bias your robot behaviour away from this circumstance.
  - Create a **`global variable`** called **`STATE`** to indicate if the robot is operating to `join line` or `follow line`.  Which state should the robot initialise into when powered on?
  - Use the `setup()` function in the template to set initial variable values, such as `STATE`.
  - Use an `if()` statement to switch the behaviour between `join line` or `follow line` depending on the global variable `STATE`.
  - Use the prior line detection function (`true` / `false`) to transition your robot between the **`join line`** behaviour, and the **`follow line`** behaviour.  
  



<hr><br><br><br><br>

<hr><br><br><br><br>

## Weighted-Measurement

So far when we have been reading and using the ground sensor, we have made `logical` decisions based on the reported value.  We have chosen through a process of `trial-and-error` experimentation a fixed `threshold` value for our conditional statements.  However, it is more desirable if the robot can determine an appropriate response **`autonomously`** without hard-coded (fixed) threshold values.  Weighted-Measurement in an example technique that will allow us to achieve this aim.  

The sensor reports numerical values so it is possible to perform some calculations.  The ground sensor reports a range of values representative of a continuous scale of reflectivity.  This range of values provides much more information from the state of reality (simulation, in this case).  

We can consider that if a sensor elements is only half across a black line, it will not report a "perfect" black value:

<p align="center">
<img width="50%" src="https://github.com/paulodowd/Webots_2021/blob/master/images/Webots_GSReflectanceHalf.png?raw=true">
</p>

Furthermore, the black line on the arena floor is sized to fit between the two outer sensor elements, which are spaced 12mm apart.  In which case, there is a continuous range of possible conditions where 1 or 2 or 3 sensing elements are either fully covered, partially covered or not covered at all.  

These properties of the sensor and environment interaction can be used to our advantage.  The numerical readings from the 3 sensor elements can be combined to approximate where the line may be under the whole ground sensor.  By analogy, we can think of this as taking the sensor readings and placing them on to a traditional weigh-scale.  The "weigh-scale" then represents a useful transformation of the data:

- The direction the scale tips indicates the relative position of the line under the robot, or **which way** to turn.
- The magnitude of tip of the scale indicates how far the line has moved from the centre of the robot, or **how much** to turn.
- when we have both `magnitude` and `direction`, we can think of this as a **`vector`**.  

<p align="center">
<img width="50%" src="https://github.com/paulodowd/Webots_2021/blob/master/images/Webots_GS_Weighted.png?raw=true">
</p>

We want to "weigh" each of the sensor readings against each other.  We could first calculate the weight of each sensing element with respect to the theoretical maximum range of the sensor ([ 0 : 1023 ]).  However, we already know that the sensor does not utilise the full range, with readings typically falling between a value of 200 and 800.  The below illustrates a typical **`sensor response`** of one sensing element:

<p align="center">
<img width="66%" src="https://github.com/paulodowd/Webots_2021/blob/master/images/Webots_CalibrationPlot.png?raw=true">
</p>

Alternatively, the weight of each sensor reading can be calculated with respect to the total activation (`summation`) of the ground sensor in that instance.  Because we expect the value of summation to vary between the uses of our ground sensor, it is useful to `normalise` the sensor values relative to the total activation, creating an output between [ 0.0 : 1.0 ].  This consistent output value range will make subsequent calculations or conditional statements more generalisable.   

<p align="center">
<img width="75%" src="https://github.com/paulodowd/Webots_2021/blob/master/images/Webots_WeightedMeasurement.png?raw=true">
</p>

With the normalised sensor values, we have the `proportional weight` of each ground sensor element (in proportion against the total activation).  Because the central sensor is between the left and right sensors, we will utilise the central sensor weight to contribute to both the left and right weights in our final weighted-measurement calculation:

<p align="center">
<img src="https://github.com/paulodowd/Webots_2021/blob/master/images/Webots_WeightedLineEq.png?raw=true">
</p>

where `gs0` is the left-most sensor, `gs1` is the central sensor, and `gs2` is the right-most sensor.  In `eq(1)` and `eq(2)`, we add half of the central sensor value, implemented by the `gain` of `0.5` in  both cases.  In `eq(3)`, we subtract the weighted-right (`w_right`) from the weighted-left (`w_left`), which provides our `error signal`, `e_line`.  

There are other methods to achieve a weighted line measurement (e.g., <a href="http://www.micromouseonline.com/2011/04/15/simpler-line-follower-sensors/">here</a>), and you are free to determine your own methods.  

We expect the `error signal` to be a value ranging between [ -1.0 : +1.0 ].  We can then use this information to decide which way to turn, and by how much.  Because the `error signal` is a value between [ -1.0 : +1.0 ], we can also use this to `proportionally control` a turning velocity.  If we hard-code (fix) a known maximum turning velocity, then values between [ -1.0 : +1.0 ] will scale the value appropriately.  













## Exercise 3: Weighted-Line Measurement

1.  Implement the weighted-line sensing as discussed above. There are still further considerations to make:
  - is your ground sensor `active low` or `active high`?   How could you invert the value of a sensor reading?  Is this necessary?
  - does the `sign` (-/+) of the error signal intuitively indicate the direction your robot should turn?  Which direction of rotation should -/+ be?
  - how can you invert the sign of the error signal?
  - **Decompose the problem**: first of all, test your robot without any forward velocity.  Is your robot able to re-centre itself over the line?  Check by manually placing your robot in different circumstances.
  - **help**: What would you expect your robot to do when it is not on a line?  Why is this?
  - **help**: The following code extract may help you to get started:

```c

void loop() {

  // Get the line error
  float e_line;
  e_line = getLineError();

  // Determine a proportional rotation speed
  float turn_velocity;
  turn_velocity = ????;  // What is a sensible maximum speed?
  turn_velocty = turn_velocity * e_line;

  // Set motor values.
  // What does "0 -" and "0 +" achieve here?
  wb_motor_set_velocity(right_motor, 0 - turn_velocity);
  wb_motor_set_velocity(left_motor, 0 + turn_velocity);
  
}

// A function to return an error signal representative
// of the line placement under the ground sensor.
float getLineError() {
  float e_line;

  // Read ground sensor, store result

  // Sum ground sensor activation

  // Normalise individual sensor readings

  // Calculated error signal
  e_line = ????;

  // Return result
  return e_line;
}

```

2. Explore the gain value for the central sensing element (described with value `0.5` above):
  - What is the effect of alternative gain values?
  - What other formulation could be used to utilise the weighted 3 sensing elements in combination?

3. Modify your solution to (1) to implement a forward velocity with your robot.
  - How fast can your robot move forwards and still reliably follow the line?
  - Which parts of the line following map present the greatest challenge to your solution?
  

4. Implement a mechanism to vary the foward velocity of your robot independently of the turning velocity:
  - Aim for high forward velocity on straight-line segments, and low forward velocity on sharp corners.
  - How can your robot measure it's own line following performance?
  - How can you convert this line performance measure into another `proportional control` mechanism for forward velocity?
